In [ ]:
#hide
import json
from pathlib import Path
from getpass import getuser, getpass
from fiscaliza.redmine import *
from fiscaliza.constants import SERVICOS
from rich.console import Console
console = Console()

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Caso não desejar utilizar um ambiente virtual basta rodar:
`pip install fiscaliza`

In [ ]:
# 
login = 'eric'
senha = 'EmD1980m#'
TESTE = True
issue = '84934'
parar_em = 'Relatando'


In [ ]:
# issue = '72088'
# detalhes = detalhar_issue(issue, login, senha)
# console.print(detalhes)

In [ ]:
d = {}

d['Classe_da_Inspecao'] = 'Técnica' # str

d['Tipo_de_Inspecao'] = 'Uso do Espectro - Monitoração' #str

d['description'] = '''Realizar análise espectral de faixas das frequência relacionadas ao Plano de Monitoração do Espectro destinado a aplicações Críticas (PMEC), aferindo a sua ocupação, nas localidades que compõem a Etapa 1 da fiscalização.'''

d['Fiscal_Responsavel'] = 'Eric Magalhaes Delgado' #str

d['Fiscais'] = ['Alessandro Vasconcelos Fernandes', 'Eric Magalhaes Delgado'] #string ou lista de strings

# Windows
d['Html'] = r'D:\OneDrive - ANATEL\Monitoramento\Processos\53500.0191952022-19\Report_2022.04.18_T13.59.18_FinalVersion.html' #str

# ['Html'] = r'D:\html_simples.html'

# #ou 

# #Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
d['Gerar_Relatorio'] = 1 # int 0 ou 1

d['Frequencia_Inicial']  = 10 #int ou float

d['Unidade_da_Frequencia_Inicial'] = 'kHz' #string

d['Frequencia_Final'] = 110 #int ou float

d['Unidade_da_Frequencia_Final'] = 'MHz' #string

d['start_date'] = '2022-04-11' #YYYY-MM-DD #string nesse formato

d['due_date'] = '2022-05-31'  #YYYY-MM-DD #string nesse formato

d['UF_Municipio'] = "BA/Salvador" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

d['Servicos_da_Inspecao'] = ['019', '205',  '230', '231', '507'] # String ou Lista de Strings

d['Qnt_de_emissoes_na_faixa'] = 70 # int

# d["Qtd_Licenciadas"] = 1

# d["Qtd_Identificadas"] = 1

d['Emissoes_nao_autorizadas'] = 24 # int

d['Horas_de_Preparacao'] = 4 # int

d['Horas_de_Deslocamento'] = 0 # int

d['Horas_de_Execucao'] = 4 # int

d['Horas_de_Conclusao'] = 16 # int

d['Latitude'] =  -23.635884 # float

d['Longitude'] = -46.654251 # float

d['Uso_de_PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Acao_de_risco_a_vida_criada'] = 'Não' # string Sim | Não

d['Impossibilidade_acesso_online'] = '0' # string '0' | '1'

d['Reservar_Instrumentos'] = '0' #string '0' = Não | '1' = 'Sim'

d['Utilizou_algum_instrumento'] = '0'

# d['notes'] = """Vemos que o nível médio medido ficou da ordem de 35.2 dB inferior e o nível máximo da ordem de 25.2 dB inferior do valor esperado pela simulação de cobertura, indicando potenciais perdas adicionais não contabilizadas pelo modelo de propagação.

# Cabe salientar que concomitantemente à monitoração efetuada no item 4 foi efetuada a demodulação do sinal de áudio e monitoração do sinal ao vivo e não foi constatado qualquer sinal de interferência, batimento ou produto de intermodulação."""

# d['notes'] = ["""[{"Banda":"87.800 - 108.000 MHz", "Licenciados":31,"NaoLicenciados":4,"NaoLicenciaveis":0, 'Espúrio': 2, "NaoIdentificada": 1, "NaoManifestada": 0},
#                 {"Banda":"470.000 - 698.000 MHz","Sinais":19,"Licenciados":19,"NaoLicenciados":0,"NaoLicenciaveis":0, 'Espúrio': 2, "NaoIdentificada": 1, "NaoManifestada": 0}]
#             """]

# No caso de uma tabela, string formatada como csv

# d['notes'] = ["""Faixa, Classe Especial, Classe A, Classe B, Classe C
#                 VHF-L,0,5,7,5
#                 VHF-H,0,12,1,0
#                 UHF,1,1,2,4
#                 FM,5,1,0,0
#                 RADCOM,0,0,0,0
#                 Outorgadas com indícios de irregularidades,1,2,3,4
#               """,
#               """[{"Banda":"87.800 - 108.000 MHz","Sinais":35,"Licenciados":31,"NaoLicenciados":4,"NaoLicenciaveis":0},
#                   {"Banda":"470.000 - 698.000 MHz","Sinais":19,"Licenciados":19,"NaoLicenciados":0,"NaoLicenciaveis":0}]
#               """]

# d['uploads'] = {'path': r'D:\OneDrive - ANATEL\Monitoramento\Processos\Diogo\Mapa3\106_1.csv', 
#                 'filename': "Lista_de_Exceção_PMEC_Etapa_2_Sorocaba", 
#                 'description': "Lista de Emissões Desmascaradas e Não Licenciadas",
#                 'content_type': '.csv'
#                }

In [ ]:
json.dump(d, Path("dados.json").open("w"))

In [ ]:
#dados = validar_dicionario(d, issue, login, senha, teste=TESTE)

In [ ]:
for k,v in dados.items():
    if k == "Html":
        v = str(v)
        v = v[:100] + '\n...\n' + v[-100:]
    console.print(f'[blue]{k} [red]-> [green]{v}')

Classe_da_Inspecao -> {'id': 89, 'value': '{"valor":"Técnica","texto":"Técnica"}'}

Tipo_de_Inspecao -> {'id': 2, 'value': '{"valor":"Uso do Espectro - Monitoração","texto":"Uso
do Espectro - Monitoração"}'}

description -> Realizar análise espectral de faixas das frequência relacionadas ao Plano de 
Monitoração do Espectro destinado a aplicações Críticas (PMEC), aferindo a sua ocupação, nas 
localidades que compõem a Etapa 1 da fiscalização.

Fiscal_Responsavel -> {'id': 25, 'value': 1262}

Fiscais -> {'id': 26, 'value': [333, 1262]}

...

Acao_de_risco_a_vida_criada -> {'id': 154, 'value': 'Não'}

Impossibilidade_acesso_online -> {'id': 450, 'value': '0'}

Reservar_Instrumentos -> {'id': 596, 'value': '0'}

Utilizou_algum_instrumento -> {'id': 598, 'value': '0'}

Coordenadas_Geograficas -> {'id': 717, 'value': '{"latitude"=>"-23.635884", 
"longitude"=>"-46.654251"}'}

In [ ]:
from fiscaliza.redmine import *
relatar_inspecao(issue, login, senha, dados=dados, teste=TESTE, parar_em=parar_em, substituir_relatorio=True)

(     ●) Atualizando...

ValueError: Não foi possível gerar o Relatório de Monitoramento

In [ ]:
#relato = relatar_inspecao(issue, login, senha, dados=dados, parar_em="Relatada")

Os mesmos passos acima podem ser efetuados no **Fiscaliza** de produção, bastando passar o argumento `teste=False` nas funções acima

In [ ]:
# inspecao = '51849'
# detalhes = detalhar_issue(inspecao, login, senha, teste=False)
# console.print(detalhes)